In [38]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

In [39]:
def get_grid(model):
    grid = np.zeros( (model.grid.width, model.grid.height) )
    for (content, x, y) in model.grid.coord_iter():
        for cont in content:
            #asignamos el color a la celda de nuestra matriz segun lo que esta contenga
            if isinstance(cont, AgentRobot):
                if cont.status == 'sin_carga':
                    grid[x][y] = 1
                else:
                    grid[x][y] = .1
            else:
                if cont.status == 'caja':
                    if cont.peso ==  1:
                        grid[x][y] = 0.9
                    if cont.peso ==  2:
                        grid[x][y] = 0.7
                    if cont.peso ==  3:
                        grid[x][y] = 0.5
                    if cont.peso ==  4:
                        grid[x][y] = 0.3
                    if cont.peso ==  5:
                        grid[x][y] = 0.1
                else:
                    grid[x][y] = 0
                
    return grid

In [118]:
class AgentRobot(Agent):
    def __init__(self, pos, model, height, width):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.new_pos = None
        self.status = 'sin_caja'
        self.height = height
        self.width = width
        self.entrega_caja = False
        
    def step(self):
        bandera = False
        if self.status == 'sin_caja':
            if self.entrega_caja == False:
                neighbours = self.model.grid.get_neighbors(self.pos, moore = True, include_center = True)
                for neighbour in neighbours:
                    if (self.pos == neighbour.pos and 
                    isinstance(neighbour, AgentCaja) and 
                    neighbour.status == 'caja' and 
                    neighbour.peso == 1):
                        self.status = 'con_caja'
                        neighbour.status = 'vacio'
                        bandera = True
                        break
            else:
                bandera = False
                self.entrega_caja = False
                
            if bandera == False:
                n = random.choice([1, 2])
                if self.borde() == 'ar_de':
                    if n == 1:
                        self.y -= 1
                    else:
                        self.x += 1
                        
                elif self.borde() == 'ar_iz':
                    if n == 1:
                        self.y += 1
                    elif n == 2:
                        self.x += 1
                    
                elif self.borde() == 'ab_de':
                    if n == 1:
                        self.y -= 1
                    elif n == 2:
                        self.x -= 1
                    
                elif self.borde() == 'ab_iz':
                    if n == 1:
                        self.y += 1
                    elif n == 2:
                        self.x -= 1
                
                elif self.borde() == 'ar':
                    n = random.choice([1, 2, 3])
                    if n == 1:
                        self.y += 1
                    elif n == 2:
                        self.x += 1
                    else:
                        self.y -= 1
                    
                elif self.borde() == 'ab':
                    n = random.choice([1, 2, 3])
                    if n == 1:
                        self.y += 1
                    elif n == 2:
                        self.x -= 1
                    else:
                        self.y -= 1
                        
                elif self.borde() == 'iz':
                    n = random.choice([1, 2, 3])
                    if n == 1:
                        self.x += 1
                    elif n == 2:
                        self.y += 1
                    else:
                        self.x -= 1
                        
                elif self.borde() == 'de':
                    n = random.choice([1, 2, 3])
                    if n == 1:
                        self.x += 1
                    elif n == 2:
                        self.y -= 1
                    else:
                        self.x -= 1
                    
                else:
                    n = random.choice([1, 2, 3, 4])
                    if n == 1:
                        self.x += 1
                    elif n == 2:
                        self.x -= 1
                    elif n == 3:
                        self.y += 1
                    else:
                        self.y -= 1
                        
        else:
            if self.x != self.model.pilas_completas:
                self.x -= 1
                
            else:
                if self.y == self.model.pilas:
                    neighbours = self.model.grid.get_neighbors(self.pos, moore = True, include_center = True)
                    for neighbour in neighbours:
                        if self.pos == neighbour.pos and isinstance(neighbour, AgentCaja):
                            self.status = 'sin_caja'
                            self.entrega_caja = True
                            if neighbour.status == 'vacio':
                                neighbour.status = 'caja'
                                neighbour.peso = 1
                            else:
                                if neighbour.peso < 4 and neighbour.peso > 0:
                                    neighbour.peso += 1
                                elif neighbour.peso == 4:
                                    neighbour.peso += 1
                                    self.model.pilas += 1
                                    if self.model.pilas == self.width:
                                        self.model.pilas_completas += 1
                                        self.model.pilas = 0
                                    
                                    
                    
                elif self.model.pilas < self.y:
                    self.y -= 1
                  
                else:
                    self.y += 1
                    
            
        self.new_pos = self.x, self.y
        self.model.grid.move_agent(self, self.new_pos)
        
    def borde(self):
        if self.x == 0 and self.y == 0:
            return 'ar_iz'
        
        elif self.x == 0 and self.y == self.width - 1:
            return 'ar_de'
        
        elif self.x == self.height - 1 and self.y == 0:
            return 'ab_iz'
        
        elif self.x == self.height - 1 and self.y == self.width - 1:
            return 'ab_de'
        
        elif self.x == 0:
            return 'ar'
        
        elif self.x == self.height - 1:
            return 'ab'
        
        elif self.y == 0:
            return 'iz'
        
        elif self.y == self.width - 1:
            return 'de'
        
        else:
            return 'centro'

In [119]:
class AgentCaja(Agent):
    def __init__(self, pos, model, status):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.status = status
        self.peso = 1

In [120]:
class Matriz(Model):

    def __init__(self, height, width, num_cajas):
        self.num_robots = 5
        self.num_cajas = num_cajas
        self.grid = MultiGrid(height, width, True)
        self.pilas = 0
        self.pilas_completas = 0
        self.fin = 0
        self.schedule = SimultaneousActivation(self)
        
            
        celdas_vacias = list(self.grid.empties)
        for i in range(5):
            celda_vacia = random.choice(celdas_vacias)
            robot = AgentRobot(celda_vacia, self, height, width)
            self.grid.place_agent(robot, celda_vacia)
            self.schedule.add(robot)
            celdas_vacias.remove(celda_vacia)
            
            
        celdas_vacias = list(self.grid.empties)    
        for celdas in range(num_cajas):
            celda_vacia = random.choice(celdas_vacias)
            caja = AgentCaja(celda_vacia, self, 'caja')
            self.grid.place_agent(caja, celda_vacia)
            self.schedule.add(caja)
            celdas_vacias.remove(celda_vacia)
            
            
        celdas_vacias = list(self.grid.empties)
        for celdas in celdas_vacias:
            caja = AgentCaja(celdas, self, 'vacio')
            self.grid.place_agent(caja, celdas)
            self.schedule.add(caja)
            
           
        self.datacollector = DataCollector(model_reporters={'Matriz': get_grid})
    
    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step() 

In [129]:
#Variables iniciales

HEIGHT = 10
WIDTH = 10
numero_cajas = 25
max_steps = 500
i = 0
#Inicualizar modelo
model = Matriz(HEIGHT, WIDTH, numero_cajas)

while(i < max_steps):
    model.step()
    i += 1


In [130]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [131]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greens')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [132]:
anim